# init

In [1]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping

import Geometry
from Geometry import Point
from camera import Camera
import draw
from inputs import input_update

# reload modules

In [221]:

importlib.reload(Clipping)
importlib.reload(Geometry)
importlib.reload(sys.modules['vec'])
importlib.reload(sys.modules['util'])
importlib.reload(sys.modules['opengl_draw_2d'])
importlib.reload(sys.modules['opengl_draw_3d'])
import Geometry

importlib.reload(sys.modules['camera'])
from camera import Camera
importlib.reload(sys.modules['inputs'])
from inputs import input_update

importlib.reload(sys.modules['draw'])
import draw



# build shapes

In [197]:
from colors import *
    
d = 3 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

cube_shape = Geometry.build_cube(d)

#draw a pair of cubes for each axis
for face, color in zip(cube_shape.faces,tess_face_colors):
    face.color = color
shapes = []
r = 2
for i in range(d):
    cube = cube_shape.copy()
    cube.update(pos = r*np.eye(d)[i])
    shapes.append(cube)
    cube = cube_shape.copy()
    cube.update(pos = -r*np.eye(d)[i])
    shapes.append(cube)
#shapes = [shapes[-2]] 
shapes.pop(2) #removes top cube
#cube_shape.transparent = True
    
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)

shape_combinations = list(itertools.combinations(shapes,2))

In [228]:
d = 3
cylinder_shape = Geometry.build_cylinder(r=1,h=2,axis=1,n_circ_pts=20)
shapes = [cylinder_shape]

# main loop

Would like draw.py to have clearer distinctions between the various spaces. The layers are
1. The d-dim local spaces that shape.verts_ref, ref_frame, and face.normal_ref etc live in
2. The d-dim world space that rotated, scaled and positioned shapes (and faces) and cameras live
3. The d-dim space rotated and translated wrt to a camera
4. The (d-1)-dim space that shapes are projected on to
5. The (d-1)-dim graphics backend (openGL)

It would be nice if we could organize code according to these layers.

Clipping happens in 2

Todo: 
- finish duocylinder
- implement cubinders, spherinders, spheres
- generalize face fuzz to all convex faces, invariant to edge order

In [24]:
import colors
import vec
def test_points(camera,pos):
    points = [ vec.Vec([np.cos(th),np.sin(th),0,0]) + pos for th in np.linspace(0,2*np.pi,200)]
    draw.draw_points(camera,points,colors.GREEN)

In [229]:
#does not draw before input / does not update camera properly?

t = 0
#size = width, height = 1100, 700
size = width, height = 1600, 900

pygame.init()

if d == 3:
    draw.init(d,size,focal=6,n_fuzz_points = 200)
    draw.view_radius = 7
    draw_view_height = 7
    draw.view_boundary = 'cylinder'
    camera = Camera(pos = Point([0,0,-10]))
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
    camera.look_at(vec.zero_vec(d))
    #camera.look_at(vec.zero_vec(d))
    #camera = Camera(pos = Point([0,0,0]))
if d == 4:
    draw.init(d,size,stereo=True,n_fuzz_points = 200)
    draw.view_boundary = 'cylinder'
    if draw.stereo:
        draw.view_radius = 4
    else:
        draw.view_radius = 6
        draw.view_height = 3
    #camera = Camera(pos = Point([0,0,0,0]))
    draw.face_scales = [0.5]
    #draw_class.stereo_view_angles = [[-30,0],[0,0]]
    #camera = Camera(pos = Point([0,0,0,0]))
    camera = Camera(pos = vec.ones_vec(d))
    camera.look_at(vec.zero_vec(d))
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(2,-1,-np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
#camera.update_rot_matrix(2,3,-np.pi/4)
draw.clipping = True
quit = False
redraw = True
timer = time.time()
frame = 0
try:
    while not quit:

        if redraw:
            draw.draw(camera,shapes)
            #test_points(camera,shapes[0].pos)
            pygame.display.flip()
            redraw = False
            
        #this is here so that we draw on the first frame
        redraw, quit = input_update(camera, shapes)
        shapes[0].update(angles=[0,time.time()]) #rotate shapes[0]
        redraw = True #redraw every loop. useful for assessing framerate


        delta_t = (time.time() - timer)
        if delta_t > 2:
        #if False:
            #print('redraw:',redraw)
            try:
                #fps = 30/(time.time() - timer)
                fps = frame/delta_t
            except:
                fps = 0
            #if fps < 60:
            print(fps,'fps')
            timer = time.time()
            frame = 0
        frame = frame+1
except:
    pygame.quit()
    print('game terminated')
    raise
pygame.quit()
print('game terminated')

init GL


C:\Users\kook\Documents\python\4d\4d-game\Clipping.py:247: RuntimeWarning: invalid value encountered in double_scalars
  tm = (-b - sqrt_discr)/(2*a)
C:\Users\kook\Documents\python\4d\4d-game\Clipping.py:248: RuntimeWarning: invalid value encountered in double_scalars
  tp = (-b + sqrt_discr)/(2*a)


78.72562523135377 fps
87.0611658957517 fps
89.9458412594555 fps
80.15028992159127 fps
89.98281568637734 fps
88.42010302770734 fps
game terminated


In [188]:
pygame.quit()

In [79]:
import vec
vec.norm(Vec([-5.55299282,  3.03433562]))

6.327947685764504

In [88]:
test= Geometry.Line(Vec([-8.38712281,  0.73238626]),Vec([-5.55299282,  3.03433562]))
print(Clipping.sphere_line_intersect(test,7,return_ts=True))
print(Clipping.clip_line_cylinder(test,7,7,axis=1))

([array([-6.66864923,  2.12817233]), array([-0.71568071,  6.96331826])], (9.883093514931366, 2.213903837983771))
u1 in 1.3871228099999993 15.38712281 Line([-8.38712281  0.73238626],[-5.55299282  3.03433562])
Line([-7.          3.92547272],[-5.55299282  3.03433562])


In [181]:
camera.frame

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [142]:
Clipping.clip_line_plane(Geometry.Line(Vec([0,0]),Vec([1,-3])),Geometry.HyperPlane(vec.one_hot(2,1),-2))

Line([0 0],[ 0.66666667 -2.        ])

In [52]:
from vec import Vec
test =Clipping.sphere_line_intersect(Geometry.Line(Vec([2,0]),Vec([-2,0])),4,return_ts=True)
print(test)

([array([4., 0.]), array([-4.,  0.])], (6.0, -2.0))


None
